# P-Value and Type of Error with Hypothesis Test

P-value is the $P(Z \ge Z_\text{Statistic})$ and if the p-value is less than the $\alpha$ then we reject the null hypothesis

If p is low, the $H_0$ must go

In [3]:
import scipy.stats as st
from ConfidenceInterval.SingleMean import SingleMean

In [2]:
1 - st.norm.cdf(1.983)

0.023683714769589748

In [8]:
2 * (1 - st.t.cdf(0.75, 8))

0.4747311849259046

In [60]:
def get_p_value(
    claimed_mean: float,
    sample_mean: float,
    sample_variance: float,
    sample_size: float,
    confidence_level: float,
    score: str,
    tails: str
) -> float:
    single_mean = SingleMean(mean=sample_mean, 
                             variance=sample_variance,
                             sample_size=sample_size,
                             score=score, 
                             confidence_level=confidence_level,
                             tails=tails)    

    test_statistic = (sample_mean - claimed_mean)/single_mean.standard_error
    critical_func = (lambda x: st.t.cdf(x, sample_size-1)) if score == "t" else lambda x: st.norm.cdf(x)
    if tails == "=":
        return 2 * (1 - critical_func(test_statistic))
    elif tails == "<=":
        return 1 - critical_func(test_statistic)
    return critical_func(test_statistic)

In [58]:
get_p_value(
    claimed_mean=60, 
    sample_mean=62.75,
    sample_variance=10**2,
    sample_size=52,
    confidence_level=0.95,
    score="z",
    tails="<="
)

0.023680743614172495

In [59]:
get_p_value(
    claimed_mean=4, 
    sample_mean=4.3,
    sample_variance=1.2**2,
    sample_size=9,
    confidence_level=0.9,
    score="t",
    tails="="
)

0.47473118492590505

## Types of Error

* Type I error is when the null hypothesis is true, but we end up rejecting the null hypothesis
    - i.e. convicting an innocent person
    - $P(\text{Type 1 Error}) = P(\text{Reject } H_0 | H_0 \text{ is true} ) = \alpha$
* Type II error is when the null hypothesis is false, but we fail to reject the null hypothesis
    - i.e. failing to convict a guilty person
    - $P(\text{Type 2 Error}) = P(\text{Fail to Reject } H_0 | H_0 \text{ is false}) = \beta$
    
    
* If $\alpha$ increases, $\beta$ decreases
* If $\alpha$ decreases, $\beta$ increases

## Power
Power of a test is the probability of correcrly rejecting a false null hypothesis.

Power = $P(\text{reject } H_0 | H_a \text{ is true}) = 1 - \beta$

In [61]:
get_p_value(claimed_mean=100,
            sample_mean=108,
            sample_variance=16**2,
            sample_size=16,
            confidence_level=0.95,
            score="z",
            tails="<=")

0.3612399686876647


0.02275013194817921

In [64]:
def β(
    claimed_mean: float,
    sample_mean: float,
    sample_variance: float,
    sample_size: float,
    confidence_level: float,
    score: str,
    tails: str
) -> float: 
    single_mean = SingleMean(mean=sample_mean, 
                         variance=sample_variance,
                         sample_size=sample_size,
                         score=score, 
                         confidence_level=confidence_level,
                         tails=tails)    
    
    if tails == "<=":
        return st.norm.cdf((claimed_mean-sample_mean)/single_mean.standard_error +
                       single_mean.critical_value)
    elif tails == ">=":
        return 1 - st.norm.cdf((claimed_mean-sample_mean)/single_mean.standard_error -
                       single_mean.critical_value)
    return st.norm.cdf((claimed_mean-sample_mean)/single_mean.standard_error +
                       single_mean.critical_value) -\
           st.norm.cdf((claimed_mean-sample_mean)/single_mean.standard_error -
                       single_mean.critical_value)
